First we setup some system stuff to handle symbolic manipulations.

In [1]:
pip install sympy # if sympy is already installed, this step can be commented out.

In [2]:
import sympy as sp
sp.init_printing()

## The problem
An ideal gas is compressed from 1 bar to 2 bar and warms up from 298 K to 373 K. Calculate the heat and work involved in the change. Since heat and work are not state functions, we have to define more precisely how we are changing things. First we will calculate for a scenario in which the pressure is changed from 1 to 2 bar isothermally and then the temperature is changed to 373 K isobarically. 

First we will define some variables and constants. These can be defined all together but it will sometimes be convenient to specify these as fully as possible so that the system knows how to treat them and here we'll keep the constants separate from the variables.

In [3]:
p,V,T=sp.symbols("p,V,T",real=True,positive=True) 
R,Cv,Cp=sp.symbols("R,Cv,Cp",real=True,positive=True,constant=True) 
ps,Ts,Vs=[],[],[] # define some variables to contain the list of beginning, intermediate, and ending pressures (p's), temperatures (T's), and volumes (V's)
Rs=[0.08314,8.314] # Define two R values. R[0] will be in Lbar/molK and R[1] will be in J/molK. We need to remember to use the appropriate one.

Next we'll define the temperatures and pressures at the beginning and end of the two steps involved in our process.

In [4]:
Ts=[298,298,373] # set up the temperatures for the beginning, intermediate, and ending points.
ps=[1,2,2] # set up the pressures for the beginning, intermediate, and ending points.

Now we can setup the equation of state. This is not strictly necessary since we could always manually manipulate the equation (especially the ideal gas equation) and input the form we want into the expression we need to calculate, but this will give us flexibility to change the equation without changing any of the following code if we so desire.

In [5]:
ig=sp.Eq(p*V,R*T) #ideal gas equation. 

Now we'll use the equation to solve for the volumes at the various points using the temperatures and pressures we declared above. We first solve the equation, ig, for V and then substitute into it the values for p, T, and R.

In [6]:
Vs=[sp.solve(ig,V)[0].subs(p,ps[i]).subs(T,Ts[i]).subs(R,Rs[0]) for i in range(len(ps))] # range(len(ps)) will make a list starting from 0 with as many members as are in ps


Now to check that the solving and substitution (and unit manipulation) went as expected, we print the results. 

In [7]:
print([Vs[i].round(2) for i in range(len(Vs))]) #print the three volumes to make sure they make sense

[24.78, 12.39, 15.51]


## 1 - in terms of dT and dp
Let's calculate the heat using $$dq=C_p dT + l_p dp$$ The first step is isothermal so we only need to calculate using the pressure term. $$l_p=-T\left(\dfrac{\partial V}{\partial T}\right)$$

In [8]:
lp=-T*sp.diff(sp.solve(ig,V)[0],T) # find lp.
q=sp.integrate(lp,(p,ps[0],ps[1])) # calculate the heat by integrating wrt pressure.
q1=q.subs(R,Rs[1]).subs(T,Ts[0]).evalf() # substitute R and T, evaluate, and save as q1.


In [9]:
print("The heat in the isothermal step is ",q1.round(2)," J/mol")


The heat in the isothermal step is  -1717.32  J/mol


The second step is isobaric so we can just use the $C_pdT$ part.

In [10]:
q=sp.integrate(Cp,(T,Ts[1],Ts[2])) # integrate Cp wrt to T.
q2=q.subs(Cp,20.785) #substitute Cp value and save as q2.


In [11]:
print("The heat in the isobaric step is ",q2.round(2), " J/mol")

The heat in the isobaric step is  1558.88  J/mol


In [12]:
q_tot1=q1+q2
print("The total heat for the two steps is ",q_tot1.round(2), " J/mol")

The total heat for the two steps is  -158.45  J/mol


## 2 - In terms of dT and dV
Let's calculate the same change using a different heat equation. $$dq=C_VdT+l_vdV$$

In [13]:
lv=T*sp.diff(sp.solve(ig,p)[0],T)
q=sp.integrate(lv,(V,Vs[0],Vs[1]))
q1=q.subs(R,Rs[1]).subs(T,Ts[0]).evalf()

print("The heat in the isothermal step is ",q1.round(2)," J/mol")


The heat in the isothermal step is  -1717.32  J/mol


Because we didn't choose a natural equation to do the calculation, neither term is zero in the next step. We will have to do an integral for both terms.

In [14]:
q_a=sp.integrate(Cv,(T,Ts[1],Ts[2]))
q_b=sp.integrate(p,(V,Vs[1],Vs[2]))*100 #100 changes pressure/volume energy to joules
q2=q_a.subs(Cv,12.471)+q_b.subs(p,ps[2])

print("The heat in the isobaric step is ",q2.round(2), " J/mol")

The heat in the isobaric step is  1558.88  J/mol


In [15]:
q_tot2=q1+q2
print("The total heat for the two steps is ",q_tot2.round(2), " J/mol")
print("The total heat using this heat equation, ",q_tot2.round(2),", should turn out to be identical to the heat using the other heat equation,",q_tot1.round(2),". If not, we've done something wrong.")

The total heat for the two steps is  -158.45  J/mol
The total heat using this heat equation,  -158.45 , should turn out to be identical to the heat using the other heat equation, -158.45 . If not, we've done something wrong.


## 3 - In terms of dp and dV
One more time with the last heat equation...  $ dq=\gamma_p dp + \gamma_V dV$. This is requires a bit more work (at least in the first step) because both p and V are changing. But we should get the same result.


In [16]:
gamv=Cp/sp.diff(sp.solve(ig,V)[0],T) # Don't worry about where these two lines come from for now. We'll figure out how to determine gamma later.
gamp=Cv/sp.diff(sp.solve(ig,p)[0],T) 

q1a=sp.integrate(gamp.subs(V,sp.solve(ig,V)[0]),(p,ps[0],ps[1]))
q1b=sp.integrate(gamv.subs(p,sp.solve(ig,p)[0]),(V,Vs[0],Vs[1])) #in order to get the right answer, we need to subsitute for p (or V) in terms of V (or p) before integrating.
print(q1a+q1b)



-0.693147180559945*Cp*T + Cv*T*log(2)


In [17]:
q1=(q1a+q1b).subs(Cv,12.471).subs(Cp,20.785).subs(T,Ts[0]).evalf()
print("The heat in the isothermal step is ",q1.round(2)," J/mol")

The heat in the isothermal step is  -1717.32  J/mol


For the isobaric case, the $dp$ term is zero but the integral in terms of V has both T and p in it. Convert $dV$ to $dT$ before integrating
$$dV=\left(\dfrac{\partial V}{\partial T}\right)dT$$

In [18]:
q=sp.integrate(gamv*sp.diff(sp.solve(ig,V)[0],T),(T,Ts[1],Ts[2])) #integrate gamma_v*(dV/dT)dT
q2=q.subs(p,sp.solve(ig,p)[0]).subs(Cp,20.785).evalf() #subsititute for p in terms of T (and V)
print("The heat in the isothermal step is ",q2.round(2)," J/mol")


The heat in the isothermal step is  1558.88  J/mol


In [19]:
q_tot3=q1+q2
print("The total heat for the two steps is ",q_tot3.round(2), " J/mol")
print("The total heat ",q_tot3.round(2)," should turn out to be identical to the heat using the other heat equations",q_tot1.round(2))

The total heat for the two steps is  -158.45  J/mol
The total heat  -158.45  should turn out to be identical to the heat using the other heat equations -158.45


## Work too!

In [26]:
w1=-sp.integrate(sp.solve(ig,p)[0],(V,Vs[0],Vs[1])) #isothermal at 298 K
print(w1)
w2=-sp.integrate(p,(V,Vs[1],Vs[2]))*100 # isobaric at 2 bar.
print(w2)

0.693147180559945*R*T
-311.775*p


In [31]:
w_tot1=(w1.subs(R,Rs[1]).subs(T,Ts[0])+w2.subs(p,ps[1]))
print(w_tot1.round(2),"J/mol")

1093.77 J/mol


In [30]:
DeltaU=w_tot1+q_tot3
print("The change in internal energy (heat + work) is ",DeltaU.round(2),"J/mol")

The change in internal energy (heat + work) is  935.32 J/mol


## Setup second problem
Let's change the order of the steps. First isobaric followed by isothermal. 
Now we'll setup the temperatures, pressures and volumes.

In [32]:
p,V,T=sp.symbols("p,V,T",real=True,positive=True)
R,Cv,Cp=sp.symbols("R,Cv,Cp",real=True,constant=True)
Ts=[298,373,373]
ps=[1,1,2]
Rs=[0.08314,8.314]
ig=sp.Eq(p*V,R*T)

Vs=[sp.solve(ig,V)[0].subs(p,ps[i]).subs(T,Ts[i]).subs(R,Rs[0]) for i in range(len(ps))]

## 1 - in terms of dT and dp
Let's calculate the heat using $$dq=C_p dT + l_p dp$$. The first step is isobaric so we only need to calculate using the pressure term. $$l_p=-T\left(\dfrac{\partial V}{\partial T}\right)$$

In [33]:
q=sp.integrate(Cp,(T,Ts[0],Ts[1]))
q1=q.subs(Cp,20.785)
print("The heat in the isobaric step is ",q1.round(2), " J/mol")



The heat in the isobaric step is  1558.88  J/mol


The second step is isothermal.

In [34]:
lp=-T*sp.diff(sp.solve(ig,V)[0],T)
q=sp.integrate(lp,(p,ps[1],ps[2]))
q2=q.subs(R,Rs[1]).subs(T,Ts[2]).evalf()
print("The heat in the isothermal step is ",q2.round(2)," J/mol")

The heat in the isothermal step is  -2149.53  J/mol


In [35]:
q_tot2=q1+q2
print("The total heat for the two steps is ",q_tot2.round(2), " J/mol")

The total heat for the two steps is  -590.66  J/mol


The two pathways have different heat and work but the same heat+work

In [41]:
w1=-sp.integrate(p,(V,Vs[0],Vs[1]))*100
w2=-sp.integrate(sp.solve(ig,p)[0],(V,Vs[1],Vs[2]))
print(w1+w2)
w_tot2=(w1+w2).subs(p,ps[0]).subs(T,Ts[2]).subs(R,Rs[1])
print("Work is",w_tot2.round(2),"J/mol")



0.693147180559945*R*T - 623.55*p
Work is 1525.98 J/mol
The change in internal energy (work plus heat) is  935.32 J/mol
Heat for path 1 (isothermal then isobaric) is  -158.45 J/mol compared to this path (isobaric then isothermal) -590.66  J/mol
Work for path 1 (isothermal then isobaric) is  1093.77 J/mol compared to this path (isobaric then isothermal) 1525.98  J/mol
Heat plus work for path 1 (isothermal then isobaric) is  935.33 J/mol compared to this path (isobaric then isothermal) 935.32  J/mol


In the first part of this exercise we calculated the heat (and work) for one sequence of paths using different equations. We got that same values for both steps of the pathway.
Now we have also calculated the same total change in state using a different sequence of steps. Note how the heat is different for these two different path sequences,  the work is different for the different path sequences, but the sum of heat and work is the same.

In [42]:
print("Heat for path 1 (isothermal then isobaric) is ",q_tot1.round(2),"J/mol compared to this path (isobaric then isothermal)", q_tot2.round(2)," J/mol")
print("Work for path 1 (isothermal then isobaric) is ",w_tot1.round(2),"J/mol compared to this path (isobaric then isothermal)", w_tot2.round(2)," J/mol")
print("Heat plus work for path 1 (isothermal then isobaric) is ",(q_tot1+w_tot1).round(2),"J/mol compared to this path (isobaric then isothermal)", (q_tot2+w_tot2).round(2)," J/mol")


Heat for path 1 (isothermal then isobaric) is  -158.45 J/mol compared to this path (isobaric then isothermal) -590.66  J/mol
Work for path 1 (isothermal then isobaric) is  1093.77 J/mol compared to this path (isobaric then isothermal) 1525.98  J/mol
Heat plus work for path 1 (isothermal then isobaric) is  935.33 J/mol compared to this path (isobaric then isothermal) 935.32  J/mol


This shows that heat and work are functions that depend on the path followed and not just on the state (T, p, and V) of the beginning and end but that the sum of heat and work depends only on the beginning and ending states. We call this sum of heat and work the internal energy change of the system. Internal energy is a path independent quantity also called a state function. State functions are a lot nicer to deal with computationally since we can use any convenient path to calculate them and use an entirely different convenient path to experimentally measure them and we can be assured that since they are state functions, the results will match. We will spend a lot of our time generating and using state functions in this course.